In [2]:
from qrisp import QuantumVariable, h, x, prepare, conjugate
from qrisp.vqe.problems.heisenberg import create_heisenberg_init_function, heisenberg_problem, create_heisenberg_hamiltonian
import networkx as nx
import numpy as np
import scipy
import matplotlib.pyplot as plt
from run_QITE import run_QITE

import pickle
from time import time
from datetime import datetime

def save_data(data, filename):  
    # Open a file for writing
    with open(filename+'.pickle', 'wb') as file:
        # Write the object to the file
        pickle.dump(data, file)

def load_data(filename):
    with open(filename+'.pickle', 'rb') as file:
        # Load the object from the file
        data = pickle.load(file)
    return data

# Investigate the ground states for inceasing magnetig field strength B

# L=4

In [3]:
L = 4
G = nx.Graph()
G.add_edges_from([(k,(k+1)%L) for k in range(L-1)]) 

J = 1
B = 3
H = create_heisenberg_hamiltonian(G, J, B)
print(H)


H_matrix = H.to_array()
eigvals, eigvecs = np.linalg.eigh(H_matrix)
idx = np.argsort(eigvals)
eigvals_sorted = eigvals[idx].real
print('Eigen energies:', eigvals_sorted)
eigvecs_sorted = eigvecs[:,idx].T
eigvecs_sorted = [vec.reshape(-1,1) for vec in eigvecs_sorted]

psi = eigvecs_sorted[0]

X(0)*X(1) + X(1)*X(2) + X(2)*X(3) + Y(0)*Y(1) + Y(1)*Y(2) + Y(2)*Y(3) + 3*Z(0) + Z(0)*Z(1) + 3*Z(1) + Z(1)*Z(2) + 3*Z(2) + Z(2)*Z(3) + 3*Z(3)
Eigen energies: [-9.82842712 -9.         -7.         -6.46410162 -4.17157288 -3.82842712
 -3.         -1.          0.46410162  1.82842712  2.17157288  3.
  5.          7.82842712  9.         15.        ]


In [4]:
from qrisp import *

qf = QuantumFloat(L)
prepare(qf, psi.transpose()[0])

qf.qs.statevector(decimals=4)

-0.2706*|7> + 0.6533*|11> - 0.6533*|13> + 0.2706*|14>

In [5]:
from qrisp import QuantumFloat, x, z, dicke_state

def U_0(qv):
    [x(qv[i]) for i in range(1,L)]
    dicke_state(qv,L)

qf = QuantumFloat(L)
U_0(qf)
qf.qs.statevector(decimals=4)

0.5*|7> + 0.5*|11> + 0.5*|13> + 0.5*|14>

# L=10

In [15]:
L = 10
G = nx.Graph()
G.add_edges_from([(k,(k+1)%L) for k in range(L-1)]) 
# nx.draw(G, with_labels=True)

In [24]:
from qrisp.operators import X, Y, Z

J = 1
B = 3
H = create_heisenberg_hamiltonian(G, J, B)
print(H)

# Define scaling factor
F = 10

# Hamiltonian simulation via first order Suzuki-Trotter formula with 5 steps
def exp_H(qv, t):
    H.trotterization(method='commuting')(qv,t/F,5)

X(0)*X(1) + X(1)*X(2) + X(2)*X(3) + X(3)*X(4) + X(4)*X(5) + X(5)*X(6) + X(6)*X(7) + X(7)*X(8) + X(8)*X(9) + Y(0)*Y(1) + Y(1)*Y(2) + Y(2)*Y(3) + Y(3)*Y(4) + Y(4)*Y(5) + Y(5)*Y(6) + Y(6)*Y(7) + Y(7)*Y(8) + Y(8)*Y(9) + 3*Z(0) + Z(0)*Z(1) + 3*Z(1) + Z(1)*Z(2) + 3*Z(2) + Z(2)*Z(3) + 3*Z(3) + Z(3)*Z(4) + 3*Z(4) + Z(4)*Z(5) + 3*Z(5) + Z(5)*Z(6) + 3*Z(6) + Z(6)*Z(7) + 3*Z(7) + Z(7)*Z(8) + 3*Z(8) + Z(8)*Z(9) + 3*Z(9)


In [25]:
H_matrix = H.to_array()
eigvals, eigvecs = np.linalg.eigh(H_matrix)
idx = np.argsort(eigvals)
eigvals_sorted = eigvals[idx].real
print('Eigen energies:', eigvals_sorted)
eigvecs_sorted = eigvecs[:,idx].T
eigvecs_sorted = [vec.reshape(-1,1) for vec in eigvecs_sorted]

psi = eigvecs_sorted[0]

qf = QuantumFloat(L)
prepare(qf, psi.transpose()[0])

qf.qs.statevector(decimals=4)

Eigen energies: [-23.93848705 -23.80492013 -22.96201964 ...  32.80422607  33.
  39.        ]


-0.0013*|255> + 0.0087*|383> - 0.0257*|447> + 0.0499*|479> - 0.0739*|495> + 0.0882*|503> - 0.085*|507> + 0.0617*|509> - 0.0225*|510> - 0.0119*|639> + 0.0561*|703> - 0.1233*|735> + 0.1923*|751> - 0.2353*|759> + 0.2298*|763> - 0.168*|765> + 0.0617*|766> - 0.0339*|831> + 0.1295*|863> - 0.2344*|879> + 0.3062*|887> - 0.309*|891> + 0.2298*|893> - 0.085*|894> - 0.0576*|927> + 0.1854*|943> - 0.2834*|951> + 0.3062*|955> - 0.2353*|957> + 0.0882*|958> - 0.0678*|975> + 0.1854*|983> - 0.2344*|987> + 0.1923*|989> - 0.0739*|990> - 0.0576*|999> + 0.1295*|1003> - 0.1233*|1005> + 0.0499*|1006> - 0.0339*|1011> + 0.0561*|1013> - 0.0257*|1014> - 0.0119*|1017> + 0.0087*|1018> - 0.0013*|1020>

In [27]:
def U_0(qv):
    print("hello")
    [x(qv[i]) for i in range(2,L)]
    dicke_state(qv,L-2)

qf = QuantumFloat(L)
U_0(qf)
qf.qs.statevector(decimals=4)

hello


0.1491*|255> + 0.1491*|383> + 0.1491*|447> + 0.1491*|479> + 0.1491*|495> + 0.1491*|503> + 0.1491*|507> + 0.1491*|509> + 0.1491*|510> + 0.1491*|639> + 0.1491*|703> + 0.1491*|735> + 0.1491*|751> + 0.1491*|759> + 0.1491*|763> + 0.1491*|765> + 0.1491*|766> + 0.1491*|831> + 0.1491*|863> + 0.1491*|879> + 0.1491*|887> + 0.1491*|891> + 0.1491*|893> + 0.1491*|894> + 0.1491*|927> + 0.1491*|943> + 0.1491*|951> + 0.1491*|955> + 0.1491*|957> + 0.1491*|958> + 0.1491*|975> + 0.1491*|983> + 0.1491*|987> + 0.1491*|989> + 0.1491*|990> + 0.1491*|999> + 0.1491*|1003> + 0.1491*|1005> + 0.1491*|1006> + 0.1491*|1011> + 0.1491*|1013> + 0.1491*|1014> + 0.1491*|1017> + 0.1491*|1018> + 0.1491*|1020>

In [17]:
O = H.hermitize().eliminate_ladder_conjugates()
commuting_groups = O.group_up(lambda a, b: a.commute(b))
#for group in commuting_groups:
#    print(group)

def trotter_step_2(qarg, t, steps):
    dt = t / steps
    #
    # 1) Forward half-step
    #
    for com_group in commuting_groups:
        with conjugate(com_group.change_of_basis)(qarg, method="commuting") as diagonal_operator:
            intersect_groups = diagonal_operator.group_up(lambda a, b: not a.intersect(b))
            for intersect_group in intersect_groups:
                for term, coeff in intersect_group.terms_dict.items():
                    term.simulate(
                        -coeff * (dt/2) * (-1),
                        qarg
                    )
    
    #
    # 2) Backward half-step (reverse order)
    #
    for com_group in reversed(commuting_groups):
        with conjugate(com_group.change_of_basis)(qarg, method="commuting") as diagonal_operator:
            intersect_groups = diagonal_operator.group_up(lambda a, b: not a.intersect(b))
            for intersect_group in intersect_groups:
                for term, coeff in intersect_group.terms_dict.items():
                    term.simulate(
                        -coeff * (dt/2) * (-1),
                        qarg
                    )

F=10

def exp_H_2(qv, t, steps=2):
    for i in range(steps):
        trotter_step_2(qv, t/F, steps)

## QITE with dicke

In [28]:
from qrisp import QuantumVariable, x, dicke_state

def U_0(qv):
    [x(qv[i]) for i in range(4,L)]
    dicke_state(qv,L-4)

results = []

results.append(run_QITE(H, U_0, exp_H_2, np.linspace(.01,5.0,20), 3, method='GC', use_statevectors=True))

In [29]:
H_matrix = H.to_array()
eigvals, eigvecs = np.linalg.eigh(H_matrix)
idx = np.argsort(eigvals)
eigvals_sorted = eigvals[idx].real
print('Eigen energies:', eigvals_sorted)
eigvecs_sorted = eigvecs[:,idx].T
eigvecs_sorted = [vec.reshape(-1,1) for vec in eigvecs_sorted]

psi = eigvecs_sorted[0]
states = results[0]["statevectors"]

fidelities = []
for phi in states:
    fidelities.append(np.abs(np.dot(psi.transpose()[0],phi))**2)

fidelities

Eigen energies: [-23.93848705 -23.80492013 -22.96201964 ...  32.80422607  33.
  39.        ]


[np.float64(7.368043378527874e-20),
 np.float64(5.2654135699976456e-17),
 np.float64(2.42824352417883e-18),
 np.float64(7.601399692046814e-09)]